- strip leading and trailing pauses
- iterate from second token to last token
    - for each pass, check previous token

In [1]:
import sys, json
from collections import *

In [2]:
with open('item_to_session_to_alignment_collapsed_pauses.json') as f:
    condensed = json.load(f)

In [3]:
def iter_dict(d):
    for k1, k2v in d.items():
        for k2, v in k2v.items():
            yield k1, k2, v

In [4]:
# strip leading and trailing pauses
stripped_i2s2a = defaultdict(lambda: defaultdict(list))
for item, session, alignment in iter_dict(condensed):
    assert(len(alignment)) > 4
    if alignment[0][0] == '<pause>':
        pause_length = alignment[0][-1]
        for idx in range(len(alignment))[1:]:
            alignment[idx][1] -= pause_length
        alignment = alignment[1:]
    if alignment[-1][0] == '<pause>':
        alignment = alignment[:-1]
    stripped_i2s2a[item][session] = alignment

In [5]:
# merge pauses with next word, prolonging length of next word
collapsed_i2s2a = defaultdict(lambda: defaultdict(list))
for item, session, alignment in iter_dict(stripped_i2s2a):
    new_alignment = []
    for i in range(len(alignment))[1:]:
        if alignment[i - 1][0] == '<pause>':
            alignment[i][1] = alignment[i - 1][1]
            alignment[i][-1] += alignment[i - 1][-1]
        else:
            new_alignment.append(alignment[i - 1])
    collapsed_i2s2a[item][session] = [*new_alignment, alignment[-1]]

In [6]:
# sanity check
for item, session, alignment in iter_dict(collapsed_i2s2a):
    for idx, item in enumerate(alignment[:-1]):
        assert item[1] + item[-1] == alignment[idx + 1][1]
        assert '<' not in item[0]

In [7]:
with open('item_to_session_to_alignment_no_pauses.json', 'x') as f:
    json.dump(collapsed_i2s2a, f, indent=2)